# BLSTM Notebook

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding, Input, TimeDistributed
from source.model.load_data import train_test_split, train_test_split_LSTM

from source.model.scoring_metrics import get_windiff, get_pk, get_k_kappa

In [2]:
# these are hyperparameters. 
# TODO: We should try to find good values for these
batch_size = 64
n_timesteps = 20
train_ratio = 0.6 # ratio of meetings the model is trained on
LSTM_units = 20

features = ['pause','speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']

datasets = """Bed002 Bed003 Bed004 Bed005 Bed006 Bed008 Bed009 Bed010 Bed011 Bed012 Bed013 Bed014 Bed015 Bed016 Bed017 Bmr001 Bmr002 Bmr005 Bmr007 Bmr009 Bmr010 Bmr011 Bmr012 Bmr013 Bmr014 Bmr018 Bmr019 Bmr021 Bmr022 Bmr024 Bmr025 Bmr026 Bmr027 Bmr029 Bns001 Bns002""".split(" ")
results_merged_path = "./results_merged_fixedf0/"

In [3]:
#load the data
X_train_df, Y_train_df, X_test_df, Y_test_df= train_test_split_LSTM(datasets, results_merged_path, n_timesteps, split=train_ratio)

print(X_train_df.shape)
print(Y_train_df.shape)
print(X_test_df.shape)
print(Y_test_df.shape)

# convert data from dataframes into numpy arrays of in a 3-D shape (samples, timesteps, features)
X_train = X_train_df.values.astype('float32').reshape(-1, n_timesteps, 5)
X_test = X_test_df.values.astype('float32').reshape(-1, n_timesteps, 5)
Y_train = Y_train_df.values.astype('float32').reshape(-1, n_timesteps, 1)
Y_test = Y_test_df.values.astype('float32').reshape(-1, n_timesteps, 1)

(20580, 5)
(20580, 1)
(31220, 5)
(31220, 1)


In [4]:
# build the model
model = Sequential()
model.add(Bidirectional(LSTM(LSTM_units, return_sequences=True), input_shape=(n_timesteps, 5)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#train the model
history=model.fit(X_train, Y_train,
           batch_size=batch_size,
           epochs=100,
        #    class_weight= {0:1, 1:10},
        #    sample_weight_mode='temporal',
           validation_data=[X_test, Y_test])

Epoch 1/100
17/17 [==============================] - 5s 78ms/step - loss: 0.6686 - accuracy: 0.6121 - val_loss: 0.5565 - val_accuracy: 0.9768
Epoch 2/100
17/17 [==============================] - 0s 16ms/step - loss: 0.4479 - accuracy: 0.9895 - val_loss: 0.3287 - val_accuracy: 0.9916
Epoch 3/100
17/17 [==============================] - 0s 23ms/step - loss: 0.2218 - accuracy: 0.9908 - val_loss: 0.1311 - val_accuracy: 0.9918
Epoch 4/100
17/17 [==============================] - 0s 18ms/step - loss: 0.0965 - accuracy: 0.9909 - val_loss: 0.0721 - val_accuracy: 0.9918
Epoch 5/100
17/17 [==============================] - 0s 16ms/step - loss: 0.0669 - accuracy: 0.9909 - val_loss: 0.0585 - val_accuracy: 0.9918
Epoch 6/100
17/17 [==============================] - 0s 16ms/step - loss: 0.0599 - accuracy: 0.9909 - val_loss: 0.0544 - val_accuracy: 0.9918
Epoch 7/100
17/17 [==============================] - 0s 16ms/step - loss: 0.0575 - accuracy: 0.9909 - val_loss: 0.0528 - val_accuracy: 0.9918
Epoch 

In [5]:
def find_threshold(predictions, Y_test):
    thresholds_to_try = [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5]
    windiffs = []
    min_windiff = 1.0
    best_threshold = 0
    for threshold in thresholds_to_try:
        pred = (predictions >= threshold).astype(float)
        windiff = get_windiff(Y_test, pred)
        windiffs.append(windiff)
        if windiff < min_windiff:
            min_windiff = windiff
            best_threshold = threshold
    print("windiffs with differnt thresholds:", windiffs)
    print("min_windiff:", min_windiff)
    print("best_threshold:", best_threshold)
    return best_threshold

In [6]:
#measure the performance
predictions = model(X_test).numpy()

predictions = predictions.flatten()

pred_eval = (predictions >= 0.5).astype(float)
Y_test = Y_test.flatten()

In [7]:
threshold = find_threshold(predictions, Y_test)
pred_eval = (predictions >= threshold).astype(float)

windiffs with differnt thresholds: [1.0, 1.0, 1.0, 1.0, 0.9907252888318356, 0.8616495507060333, 0.7036585365853658, 0.5970474967907574, 0.5242618741976893, 0.469191270860077, 0.44589216944801024, 0.4378369704749679, 0.42586649550706035, 0.4265725288831836, 0.39971116816431324, 0.40070603337612326, 0.3933247753530167, 0.39120667522464697]
min_windiff: 0.39120667522464697
best_threshold: 0.5


In [8]:
print("predictions:", predictions)
print("pred_eval:", pred_eval)
print("# boundaries predicted:", np.count_nonzero(pred_eval))

predictions: [0.00624905 0.0062713  0.00646483 ... 0.01037796 0.0133741  0.01399104]
pred_eval: [0. 0. 0. ... 0. 0. 0.]
# boundaries predicted: 12


In [9]:
windiff = get_windiff(Y_test, predictions)
pk = get_pk(Y_test, predictions, k=58)
k_k = get_k_kappa(Y_test, predictions)
print("windiff:", windiff)
print("pk:", pk)
print("K_k ", k_k)
print("windiff_classifier:", get_windiff(Y_test, pred_eval))
print("pk_classifier:", get_pk(Y_test, pred_eval))
print("K_k_classifier ", get_k_kappa(Y_test, pred_eval))

windiff: 0.3910462130937099
pk: 0.3777556717902641
K_k  0.3578392621870882
windiff_classifier: 0.39120667522464697
pk_classifier: 0.3872593068035943
K_k_classifier  0.013986523780164134
